In [ ]:
from scipy.special import digamma, polygamma
import numpy as np

<h1>3-1. Latent Dirichlet Allocation<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data:-Reuters" data-toc-modified-id="Data:-Reuters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data: Reuters</a></span></li><li><span><a href="#Model:-Smoothed-LDA" data-toc-modified-id="Model:-Smoothed-LDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model: Smoothed LDA</a></span><ul class="toc-item"><li><span><a href="#Variational-EM" data-toc-modified-id="Variational-EM-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Variational EM</a></span><ul class="toc-item"><li><span><a href="#E-step" data-toc-modified-id="E-step-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>E-step</a></span></li><li><span><a href="#M-step" data-toc-modified-id="M-step-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>M-step</a></span></li></ul></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span></li></ul></div>

## Data: Reuters

Reuters is a multi-class, multi-label dataset.

* 90 classes
* 10788 documents
    * 7769 training documents
    * 3019 testing documents

In [ ]:
from nltk.corpus import reuters
from nltk.corpus import stopwords

* train-test split
: The data is already splitted. Just sort it out.

In [ ]:
stops = stopwords.words("english")

In [ ]:
reuters.words()[:10]

In [ ]:
trainset, testset = [], []
vocab = []
for file_id in reuters.fileids():
    if file_id.startswith("train"):
        trainset.append([w for w in reuters.words(file_id) if w not in stops])
        vocab += trainset[-1]
    else:
        testset.append([w for w in reuters.words(file_id) if w not in stops])

In [6]:
vocab = set(vocab)
word_to_ix = {k: v for v, k in enumerate(vocab)}

In [7]:
def seq_to_ix(seq, vocab=vocab):
    # len(vocab), which is the last index, is for the <unk> (unknown) token
    unk_idx = len(vocab)
    return np.array(list(map(lambda w: word_to_ix.get(w, unk_idx), seq)))

data = {
    "train": list(map(seq_to_ix, trainset)),
    "test": list(map(seq_to_ix, testset))
}

In [8]:
data["train"][0][:5]

array([14373,  9509, 14991, 24125,  2348])

## Model: Smoothed LDA

For each document $\mathbf{w}$ in a corpus $D$, generate

$$
N \sim \mathcal{P}(\xi) \\
\beta \sim \text{Dir}(\lambda) \\
\theta \sim \text{Dir}(\alpha)
$$

and for $n = 1, \cdots, N$, generate

$$
z_n \sim \text{Multi}(\theta) \\
w_n \sim P(w_n | z_n, \beta)
$$

where $\beta \in \mathbb{R}^{k \times V}$, $\beta_{ij} = P(w^j = 1| z^i = 1)$.

* $\alpha, \eta$: Dirichlet hyperparameters.
* $\beta$: Unsmoothed multinomial hyperparameter.
* $N$: The number of words in the document. (ancillary variable)
* $\theta$: A topic mixture.
* (For $i$ in $1\cdots N$)
  * $z_n$: A topic variable.
  * $w_n$: A generated word.

### Variational EM

#### E-step

Let $\phi_d \in \mathbb{R}^{N \times k}, \gamma_d \in \mathbb{R}^k, \lambda \in \mathbb{R}^{k \times V}$ be variational parameters for $\alpha, \beta, \eta$.  
Suppose further that for $\beta \in \mathbb{R}^{k \times V}$, $\beta_i^0 \sim \text{Dir}(\lambda^0)$ where $\lambda_i^0 = \eta$ for all $i$.

For a document $\mathbf{w}_d$, $d = 1,\cdots,M$,

1. initialize $\phi_{dni}^0 := 1/k$ for all $i,n$.
2. initialize $\gamma_{di} := \alpha_i + N/k$ for all $i$.
3. **repeat until** convergence
    1. for $n=1$ to $N$
        1. for $i=1$ to $k$
            1. $\phi_{dni}^{t+1} := \beta_{iw_n} \exp\left(\Psi(\gamma_{di}^t) - \Psi(\sum_{j=1}^k \gamma_{dj}^t)\right)$
            1. for $j=1$ to $V$
                1. $\lambda_{ij} = \eta + \sum_{d=1}^M \sum_{n=1}^{N_d} \phi_{dni} w_{dn}^j$
        2. normalize $\phi_{dn}^{t+1}$ to sum to 1
    2. $\gamma_d^{t+1} := \alpha + \sum_{n=1}^N \phi_{dn}^{t+1}$
    
where $\Psi$ is the first derivative of the $\log\Gamma$ function.

In [9]:
V = len(vocab)
k = 90  # number of topics
N = np.array([len(doc) for doc in data["train"]])
M = len(data["train"])

print(f"V: {V}\nk: {k}\nN: {N}\nM: {M}")

V: 35105
k: 90
N: [472 189  96 ...  23  91  50]
M: 7769


In [10]:
# initialize α, β
α = np.ones(k)
β = np.ones((k, V)) / V
η = np.ones((k, V)) / V

print(f"α: dim {α.shape}\nβ: dim {β.shape}\nη: dim {η.shape}")

α: dim (90,)
β: dim (90, 35105)
η: dim (90, 35105)


In [11]:
# initialize ϕ, γ
## ϕ: (M x max(N) x k) arrays with zero paddings on the right
γ = α + np.ones((M, k)) * N.reshape(-1, 1) / k

ϕ = np.ones((M, max(N), k))
for m, N_d in enumerate(N):
    ϕ[m, N_d:, :] = 0  # zero padding for vectorized operations

λ = η + np.ones((k, V)) * M / V

print(f"γ: dim {γ.shape}\nϕ: dim ({len(ϕ)}, N_d, {ϕ[0].shape[1]})\nλ: dim {λ.shape}")

γ: dim (7769, 90)
ϕ: dim (7769, N_d, 90)
λ: dim (90, 35105)


In [12]:
words = data["train"]

In [ ]:
from tqdm import tqdm
from multiprocessing import Pool

def inner_sum2(m):
    return (np.array([words[m] == j for j in range(V)]) @ ϕ[m, :N[m], :]).T

with Pool(16) as p:
    res = list(tqdm(p.imap(inner_sum2, range(M)), total=M))

 12%|█▏        | 914/7769 [01:13<33:03,  3.46it/s]

In [208]:
from tqdm import tqdm

def inner_sum(m, j):
    return (words[m] == j) @ ϕ[m, :N[m], :]

def minorize(words, smooth=True):
    """
    Minorize the joint likelihood function via variational inference.
    This is the E-step of variational EM algorithm for (smoothed) LDA.
    """
    global ϕ, γ
    if smooth:
        global λ
    else:
        global β
    
    # optimize ϕ
    for m in tqdm(range(M)):
        if smooth:
            ϕ[m, :N[m], :] = np.exp(digamma(λ[:, words[m]]) - digamma(λ[:, :].sum(axis=1)).reshape(-1, 1) \
                                + (digamma(γ[m, :]) - digamma(γ[m, :].sum())).reshape(-1, 1)).T
        else:
            ϕ[m, :N[m], :] = (β[:, words[m]] * np.exp(digamma(γ[m, :]) - digamma(γ[m, :].sum())).reshape(-1, 1)).T
                    
        # Normalize phi
        ϕ[m, :N[m], :] / ϕ[m, :N[m], :].sum(axis=1).reshape(-1, 1)
    
    # optimize γ
    γ = α + np.array(list(map(lambda x: x.sum(axis=0), ϕ)))
    
    # optimize λ
    if smooth:
        λ[:] = η
        for m in tqdm(range(M)):
            λ += (np.array([words[m] == j for j in range(V)]) @ ϕ[m, :N[m], :]).T
    
        return φ, γ, λ
    else:
        return ϕ, γ

#### M-step

$$
\beta_{ij} \propto \sum_{d=1}^M \sum_{n=1}^N \phi_{dni} \mathbf{w}_{dn}^j
$$

$\alpha$ is updated via Newton-Raphson method:

$$
\frac{\partial L}{\partial \alpha_i} 
  = M\left( \Psi\left(\sum_{j=1}^k \alpha_j\right) - \Psi(\alpha_i) \right)
    - \sum_{d=1}^M \left( \Psi(\gamma_{di}) - \Psi\left(\sum_{j=1}^k \gamma_{dj}\right) \right) \\
\frac{\partial^2 L}{\partial \alpha_i \alpha_j} = M \left( \Psi'\left(\sum_{j=1}^k \alpha_j\right) - \delta(i,j) \Psi'(\alpha_i) \right)
$$

where $\delta(i,j) = 1$ if $i=j$, $0$ otherwise.

In [209]:
import warnings

def update_(var, vi_var, max_iter, tol):
    """
    From appendix A.2 of Blei et al., 2003.
    For hessian with shape `H = diag(h) + 1z1'`
    
    To update alpha, input var=alpha and vi_var=gamma.
    To update eta, input var=eta and vi_var=lambda.
    """
    for _ in tqdm(range(max_iter)):
        var_old = var
        
        # g: gradient 
        digamma_sum = digamma(vi_var.sum(axis=1)).reshape(-1, 1)
        g = M * (digamma(var.sum()) - digamma(var)) \
            + (digamma(vi_var) - digamma_sum).sum(axis=0)
        
        # H = diag(h) + 1z1'
        h = -M * polygamma(1, var)       # h: Hessian diagonal component
        z = M * polygamma(1, var.sum())  # z: Hessian constant component
        c = (g / h).sum() / (1./z + (1./h).sum())

        #  Update alpha
        var = var - (g - c) / h
        
        #  Check convergence
        if np.mean((var - var_old) ** 2) < tol ** 2:
            break
    
    warnings.warn(f"max_iter={max_iter} reached: values might not be optimal.")
    
    return var

In [210]:
def maximize(words, max_iter=1000, tol=1e-3, smooth=True):
    """
    maximize the lower bound of the likelihood.
    This is the M-step of variational EM algorithm for (smoothed) LDA.
    
    update of α follows from appendix A.2 of Blei et al., 2003.
    """
    global α, γ
    if smooth:
        global η, λ
    else:
        global β, ϕ
    
    # update α
    α = update_(α, γ, max_iter, tol)
    
    if smooth:
        # update η
        η = update_(η, λ, max_iter, tol)
        
        return α, η
    
    else:
        # update β
        for j in range(V):
            β[:, j] = np.array([inner_sum(m, j) for m in range(M)]).sum(axis=1)
        β /= β.sum(axis=1)
    
        return α, β

## Training

In [212]:
MAX_ITER = 2
for i in range(MAX_ITER):
    print(i)
    print(""*5, "MINORIZE", "="*5)
    φ, γ, λ = minorize(words)
    
    print("="*5, "MAXIMIZE", "="*5)
    α, η = maximize(words)

  2%|▏         | 171/7769 [00:00<00:08, 869.50it/s]

0
 MINORIZE =====


  7%|▋         | 551/7769 [00:39<08:42, 13.81it/s]


KeyboardInterrupt: 